In [36]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-aug-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-aug-2022/train.csv
/kaggle/input/tabular-playground-series-aug-2022/test.csv
/kaggle/input/pickle/model3.sav
/kaggle/input/pickle/model1.sav
/kaggle/input/pickle/model2.sav


In [37]:
%%capture
!pip install feature-engine

In [38]:
!rm 109550116submission.csv 109550116submission_no_fold.csv 109550116submission_kfold.csv

In [39]:
#===============import===============#
import os
from pathlib import Path
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LogisticRegression ,HuberRegressor
from sklearn.metrics import roc_auc_score
from sklearn.impute import KNNImputer
from feature_engine.encoding import WoEEncoder
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
TARGET = "failure"
#usepickle=0

In [40]:
#===============load data===============#
def read(path):
    data_dir = Path(path)
    train = pd.read_csv(data_dir / "train.csv")
    test = pd.read_csv(data_dir / "test.csv")
    return train, test

train, test = read("../input/tabular-playground-series-aug-2022")
target = train["failure"]
train.drop("failure", axis=1, inplace=True)

In [41]:
#===============preprocessing===============#
def preprocessing(df_train, df_test):
    data = pd.concat([df_train, df_test])
    data["m3_missing"] = data["measurement_3"].isnull().astype(np.int8)
    data["m5_missing"] = data["measurement_5"].isnull().astype(np.int8)
    feature = [f for f in df_test.columns if f.startswith("measurement") or f == "loading"]

    full = {}
    full["measurement_17"] = {
        "A": ["measurement_5", "measurement_6", "measurement_8"],
        "B": ["measurement_4", "measurement_5", "measurement_7"],
        "C": ["measurement_5", "measurement_7", "measurement_8", "measurement_9"],
        "D": ["measurement_5", "measurement_6", "measurement_7", "measurement_8"],
        "E": ["measurement_4", "measurement_5", "measurement_6", "measurement_8"],
        "F": ["measurement_4", "measurement_5", "measurement_6", "measurement_7"],
        "G": ["measurement_4", "measurement_6", "measurement_8", "measurement_9"],
        "H": ["measurement_4","measurement_5","measurement_7","measurement_8","measurement_9"],
        "I": ["measurement_3", "measurement_7", "measurement_8"],
    }

    col = [col for col in df_test.columns if "measurement" not in col] + ["loading","m3_missing","m5_missing",]
    a = []
    b = []
    for x in range(3, 17):
        corr = np.absolute(data.drop(col, axis=1).corr()[f"measurement_{x}"]).sort_values(ascending=False)
        a.append(np.round(np.sum(corr[1:4]), 3))  
        b.append(f"measurement_{x}")
    c = pd.DataFrame()
    c["Selected columns"] = b
    c["correlation total"] = a
    c = c.sort_values(by="correlation total", ascending=False).reset_index(drop=True)
    #print(c.head(10))

    for i in range(10):
        m_col = ("measurement_" + c.iloc[i, 0][12:])  
        fill_dict = {}
        for x in data.product_code.unique():
            corr = np.absolute(data[data.product_code == x].drop(col, axis=1).corr()[m_col]).sort_values(ascending=False)
            #print(corr)
            m_col_dic = {}
            m_col_dic[m_col] = corr[1:5].index.tolist()
            fill_dict[x] = m_col_dic[m_col]
        full[m_col] = fill_dict
    feature = [f for f in data.columns if f.startswith("measurement") or f == "loading"]
    n_col = [col for col in df_train.columns if df_train[col].isnull().sum() != 0]
    for code in data.product_code.unique():
        for m_col in list(full.keys()):
            tmp = data[data.product_code == code]
            column = full[m_col][code]
            ttrain = tmp[column + [m_col]].dropna(how="any")
            #print(tmp_train)
            ttest = tmp[(tmp[column].isnull().sum(axis=1) == 0) & (tmp[m_col].isnull())]
            #print(tmp_test)
            model = HuberRegressor(epsilon=1.9,max_iter=3000)
            model.fit(ttrain[column], ttrain[m_col])
            data.loc[(data.product_code == code) & (data[column].isnull().sum(axis=1) == 0)& (data[m_col].isnull()),m_col,] = model.predict(ttest[column])

        NA = data.loc[data["product_code"] == code, n_col].isnull().sum().sum()
        model1 = KNNImputer(n_neighbors=3)
        data.loc[data.product_code == code, feature] = model1.fit_transform(data.loc[data.product_code == code, feature])

    data["measurement_avg"] = data[[f"measurement_{i}" for i in range(3, 17)]].mean(axis=1)
    df_train = data.iloc[: df_train.shape[0], :]
    df_test = data.iloc[df_train.shape[0] :, :]
    #print(data)
    #print(df_train)
    #print(df_test)
    woe_encoder = WoEEncoder(variables=["attribute_0"])
    woe_encoder.fit(df_train, target)
    df_train = woe_encoder.transform(df_train)
    df_test = woe_encoder.transform(df_test)
    features = ["loading","attribute_0","measurement_17","measurement_0","measurement_1","measurement_2","m3_missing","m5_missing","measurement_avg"]
    return df_train, df_test, features

train, test, FEATURES = preprocessing(train, test)
train["failure"] = target

<div style="background-color:rgba(255, 0, 100, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">3 vs 2 fold</h1>
</div>

In [42]:
#===============training===============#
'''def train_model(df,test,FEATURES,TARGET):
    final_test_predictions = []
    test = test[FEATURES].copy()

    folds_dict = {f'Fold 1': [['C', 'D', 'E'], ['A', 'B']], 
               'Fold 2': [['B', 'D', 'E'], ['A', 'C']],
               'Fold 3': [['B', 'C', 'E'], ['A', 'D']],
               'Fold 4': [['B', 'C', 'D'], ['A', 'E']],
               'Fold 5': [['A', 'D', 'E'], ['B', 'C']],
               'Fold 6': [['A', 'C', 'E'], ['B', 'D']],
               'Fold 7': [['A', 'C', 'D'], ['B', 'E']],
               'Fold 8': [['A', 'B', 'E'], ['C', 'D']],
               'Fold 9': [['A', 'B', 'D'], ['C', 'E']],
               'Fold 10': [['A', 'B', 'C'], ['D', 'E']]} 
   
    for fold in folds_dict.keys():
        print(f'##### {fold} #####')
        xtest = test.copy()
        #print(df)
        xtrain, ytrain = df[df['product_code'].isin(folds_dict[fold][0])][FEATURES].values, df[df['product_code'].isin(folds_dict[fold][0])][TARGET].values
        xvalid, yvalid = df[df['product_code'].isin(folds_dict[fold][1])][FEATURES].values,  df[df['product_code'].isin(folds_dict[fold][1])][TARGET].values
        #print(xtrain)
        model = LogisticRegression(max_iter=200, C=0.0001, penalty="l2", solver="newton-cg")
        model.fit(xtrain,ytrain)
        preds_valid = model.predict_proba(xvalid)[:, 1]
        test_preds = model.predict_proba(xtest.values)[:, 1]
        #print(test_preds)
        final_test_predictions.append(test_preds)
        fold_score = roc_auc_score(yvalid, preds_valid)  
        print(fold_score)
    return (model,final_test_predictions)

#print(train)
filename = 'model1.sav'
if(usepickle==0):
    (model,final_test_preds) = train_model(train,test,FEATURES,TARGET)
    pickle.dump(model, open(filename, 'wb'))
    #final_test_preds.append(model.predict_proba(test[FEATURES].values)[:, 1])'''
final_test_preds=[]
filename = '/kaggle/input/pickle/model1.sav'
loaded_model = pickle.load(open(filename, 'rb'))
final_test_preds.append(loaded_model.predict_proba(test[FEATURES].values)[:, 1])

In [43]:
#===============submission===============#
submission_df = test[["id"]].copy().astype(int)
submission_df[TARGET] = np.mean(np.column_stack(final_test_preds), axis=1)
submission_df.to_csv("109550116submission.csv", index=False)

 
<div style="background-color:rgba(255, 0, 100, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">K-fold</h1>
</div>
 

In [44]:
#===============training===============#
'''def train_model1(df,test,FEATURES,TARGET):
    final_test_predictions = []
    test = test[FEATURES].copy()

    folds_dict = {f'Fold 1': [['B','C', 'D', 'E'], ['A']], 
               'Fold 2': [['A','C','D', 'E'], ['B']],
               'Fold 3': [['A','B','C', 'E'], ['D']],
               'Fold 4': [['A', 'B','D','E'], ['C']],
               'Fold 5': [['A', 'B', 'C','D'], ['E']]} 
   
    for fold in folds_dict.keys():
        print(f'##### {fold} #####')
        xtest = test.copy()
        xtrain, ytrain = df[df['product_code'].isin(folds_dict[fold][0])][FEATURES].values, df[df['product_code'].isin(folds_dict[fold][0])][TARGET].values
        xvalid, yvalid = df[df['product_code'].isin(folds_dict[fold][1])][FEATURES].values,  df[df['product_code'].isin(folds_dict[fold][1])][TARGET].values
        #print(xtrain)
        model = LogisticRegression(max_iter=1000, C=0.0001, penalty="l2", solver="newton-cg")
        model.fit(xtrain,ytrain)
        preds_valid = model.predict_proba(xvalid)[:, 1]
        test_preds = model.predict_proba(xtest.values)[:, 1]
        #print(test_preds)
        final_test_predictions.append(test_preds)
        fold_score = roc_auc_score(yvalid, preds_valid)  
        print(fold_score)
    return (model)

#print(train)

filename = 'model2.sav'
if(usepickle==0):
    (model_kfold) = train_model1(train,test,FEATURES,TARGET)
    pickle.dump(model_kfold, open(filename, 'wb'))
    #final_test_preds1.append(model_kfold.predict_proba(test[FEATURES].values)[:, 1])'''
final_test_preds1=[]
filename = '/kaggle/input/pickle/model2.sav'
loaded_model = pickle.load(open(filename, 'rb'))
final_test_preds1.append(loaded_model.predict_proba(test[FEATURES].values)[:, 1])

In [45]:
#===============submission===============#
submission_kfold_df = test[["id"]].copy().astype(int)
submission_kfold_df[TARGET] = np.mean(np.column_stack(final_test_preds1), axis=1)
submission_kfold_df.to_csv("109550116submission_kfold.csv", index=False)

<div style="background-color:rgba(255, 0, 100, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">No-fold</h1>
</div>

In [46]:
#===============training===============#
'''def train_model2(df,test,FEATURES,TARGET):
    final_test_predictions = []
    test = test[FEATURES].copy()

    
    #print(f'########################## {fold} ##########################')
    xtest = test.copy()
    
    xtrain, xvalid = train_test_split(df, random_state=777, train_size=0.8)
    #print(xtrain)
    ytrain = xtrain[TARGET]
    #print(ytrain)
    yvalid = xvalid[TARGET]
    xtrain = xtrain[FEATURES]
    xvalid = xvalid[FEATURES]
    
    model = LogisticRegression(max_iter=1000, C=0.0001, penalty="l2", solver="newton-cg")
    model.fit(xtrain,ytrain)
    preds_valid = model.predict_proba(xvalid)[:, 1]
    test_preds = model.predict_proba(xtest.values)[:, 1]
    #print(test_preds)
    final_test_predictions.append(test_preds)
    fold_score = roc_auc_score(yvalid, preds_valid)  
    print(fold_score)
    return (model,final_test_predictions)

#print(train)
#(model_no_fold,final_test_preds2) = train_model(train,test,FEATURES,TARGET)

filename = 'model3.sav'
if(usepickle==0):
    (model_no_fold,final_test_preds2) = train_model2(train,test,FEATURES,TARGET)
    pickle.dump(model_no_fold, open(filename, 'wb'))'''
final_test_preds2=[]
filename = '/kaggle/input/pickle/model3.sav'
loaded_model = pickle.load(open(filename, 'rb'))
final_test_preds2.append(loaded_model.predict_proba(test[FEATURES].values)[:, 1])

In [47]:
#===============submission===============#
submission_no_fold_df = test[["id"]].copy().astype(int)
submission_no_fold_df[TARGET] = np.mean(np.column_stack(final_test_preds2), axis=1)
submission_no_fold_df.to_csv("109550116submission_no_fold.csv", index=False)